# FloPy

### Zonbud_util Example

This notebook demonstrates how to use the functions included in the zonbud_util module to create a zonfile and run ZoneBudget.

First set the path and import the required packages. The flopy path doesn't have to be set if you install flopy from a binary installer. If you want to run this notebook, you have to set the path to your own flopy path.

In [1]:
%matplotlib inline
import sys
import os
import platform
import flopy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#Set name of MODFLOW exe
#  assumes executable is in users path statement
version = 'mf2005'
exe_name = 'mf2005'
if platform.system() == 'Windows':
    exe_name = 'mf2005.exe'
mfexe = exe_name


#Set name of ZoneBudget exe
#  assumes executable is in users path statement
exe_name = 'zonbud'
if platform.system() == 'Windows':
    exe_name = 'zonbud.exe'
zbexe = exe_name

modelpath = os.path.join('..', 'data', 'zonbud_util_example')

#make sure modelpth directory exists
if not os.path.exists(modelpath):
    os.makedirs(modelpath)

### Load and Run an Existing Model
A model called the "Freyberg Model" is located in the modelpath folder.  In the following code block, we load and run the model.  For this to work properly, the MODFLOW-2005 executable (mf2005) must be in the path.  We verify that it worked correctly by checking for the presence of freyberg.hds and freyberg.cbc.

In [2]:
ml = flopy.modflow.Modflow.load('freyberg.nam', model_ws=modelpath, 
                                exe_name=mfexe, version=version)
ml.write_input()
success, buff = ml.run_model()
if not success:
    print ('Something bad happened.')
files = ['freyberg.hds', 'freyberg.cbc']
for f in files:
    if os.path.isfile(os.path.join(modelpath, f)):
        msg = 'Output file located: {}'.format(f)
        print (msg)
    else:
        errmsg = 'Error. Output file cannot be found: {}'.format(f)
        print (errmsg)

FloPy is using the following executable to run the model: mf2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.11.00 8/8/2013                        

 Using NAME file: freyberg.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2016/03/09 18:29:40

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2016/03/09 18:29:40
 Elapsed run time:  0.030 Seconds

  Normal termination of simulation
Output file located: freyberg.hds
Output file located: freyberg.cbc


### Create a Zonfile and Run ZoneBudget
We create a simple numpy integer-array to define some zones for which we are interested in getting cell-by-cell budget information. The numpy array is written to a file which can then be read by ZoneBudget.

In [3]:
# Define some very interesting zones
zon = np.zeros((ml.dis.nrow, ml.dis.ncol), np.int)  # 40 rows by 20 cols
zon[:20, :10] = 1
zon[:20, 10:] = 2
zon[20:, :10] = 3
zon[20:, 10:] = 4

# Now lets write this to a file
out_zonfile = os.path.join(modelpath, 'zbud.zon')
flopy.utils.write_zonfile(zon, out_zonfile)

# Run ZoneBudget
cbc_file = os.path.join(modelpath, 'freyberg.cbc')
listing_file = os.path.join(modelpath, 'zbud csv')
flopy.utils.run_zonbud(zonbud_exe=zbexe, zonefile=out_zonfile, cbcfile=cbc_file, listingfile=listing_file)

###Look at ZoneBudget Output
We can now see that ZoneBudget aggregated cell-by-cell flows for our zones. Yay.

In [4]:
zbud = pd.read_csv(os.path.join(modelpath, 'zbud.csv'), skiprows=1)
print(zbud[zbud.columns[:-1]].dropna(how='all').to_string(index=False))

                     ZONE   1          ZONE   2          ZONE   3          ZONE   4      
                       IN                IN                IN                IN          
    CONSTANT HEAD      0.000000E+00      0.000000E+00      0.000000E+00      0.000000E+00
    RIVER LEAKAGE      0.000000E+00      2.408758E-03      0.000000E+00      0.000000E+00
         RECHARGE      1.530000E-02      2.000000E-02      1.590000E-02      1.830000E-02
    FROM ZONE   1      0.000000E+00      1.334857E-02      2.696172E-03      0.000000E+00
    FROM ZONE   2      0.000000E+00      0.000000E+00      0.000000E+00      1.567713E-03
    FROM ZONE   3      7.447382E-04      0.000000E+00      0.000000E+00      1.337981E-02
    FROM ZONE   4      0.000000E+00      2.727445E-04      0.000000E+00      0.000000E+00
 Total IN              1.604474E-02      3.603007E-02      1.859617E-02      3.324752E-02
                      OUT               OUT               OUT               OUT          
    CONSTA